In [ ]:
!pip install transformers torch datasets scikit-learn

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import BertTokenizer
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn.functional as F
import torch.nn.init as init
from transformers import AutoModel, AutoTokenizer
from datasets import load_dataset
from sklearn.metrics import cohen_kappa_score
from tqdm import tqdm

In [4]:
train_df = pd.read_csv('/kaggle/input/dress-d/train.csv', index_col=0)
val_df = pd.read_csv('/kaggle/input/dress-d/val.csv', index_col=0)
test_df = pd.read_csv('/kaggle/input/dress-d/test.csv', index_col=0)

In [5]:
class EllipseDataset(Dataset):
    def __init__(self, dataframe, tokenizer):
        self.df = dataframe
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        text = row["full_text"].to_list()



        scores = torch.tensor(row[["Overall",
                                   "Cohesion",
                                   "Syntax",
                                   "Vocabulary",
                                   "Phraseology",
                                   "Grammar",
                                   "Conventions"]
                                  ].values, dtype=torch.float32)


        inputs = self.tokenizer(text,
                                padding="max_length",
                                truncation=True,
                                return_tensors="pt",
                                max_length=512)



        return {
            "input_ids": inputs["input_ids"],
            "attention_mask": inputs["attention_mask"],
            "scores": scores
        }

In [6]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-base", use_fast=False)
special_token = "\n\n"
tokenizer.add_special_tokens({'additional_special_tokens': [special_token]})

train_dataset = EllipseDataset(train_df.reset_index(drop=True), tokenizer)
val_dataset = EllipseDataset(val_df.reset_index(drop=True), tokenizer)
test_dataset = EllipseDataset(test_df.reset_index(drop=True), tokenizer)

In [7]:
class DebertaNHeads(nn.Module):
    def __init__(self, model_name="microsoft/deberta-v3-base", num_aspects=7):
        super().__init__()
        self.deberta = AutoModel.from_pretrained(model_name)
        self.deberta.resize_token_embeddings(len(tokenizer))
        hidden_size = self.deberta.config.hidden_size

        self.num_aspects = num_aspects
        
        self.regression_heads = nn.ModuleList([
            nn.Linear(hidden_size, 1) for _ in range(num_aspects)
        ])
        self.sigmoid = nn.Sigmoid()
        self.dropout = nn.Dropout(0.2)
        
        self._init_weights()

    def _init_weights(self):
        for head in self.regression_heads:
            init.xavier_uniform_(head.weight)
            if head.bias is not None:
                init.zeros_(head.bias)

    def forward(self, input_ids, attention_mask=None, token_type_ids=None):
        outputs = self.deberta(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0, :]
        pooled_output = self.dropout(pooled_output)
        regression_outputs = []

        for i in range(self.num_aspects):
            regression_outputs.append(self.sigmoid(self.regression_heads[i](pooled_output)))

        regression_outputs = torch.cat(regression_outputs, dim=1)

        return regression_outputs


In [10]:
model = DebertaNHeads()

2025-05-16 23:23:08.427780: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747437788.451324     145 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747437788.458343     145 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [8]:
from transformers import get_scheduler
from torch.optim.lr_scheduler import CosineAnnealingLR

In [11]:
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8)
test_loader = DataLoader(test_dataset, batch_size=8)

num_training_steps = len(train_loader) * 10

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = optim.AdamW(model.parameters(), lr=4e-5)
scheduler = CosineAnnealingLR(optimizer, T_max=num_training_steps)

In [12]:
def combined_loss(reg_preds, reg_labels):
    regression_loss = nn.BCELoss()
    total_reg_loss = 0
    total_reg_loss += regression_loss(reg_preds, reg_labels)

    return total_reg_loss

In [13]:
def evaluate_qwk_for_aspects(class_preds, class_labels, aspects=['Overall', 'Cohesion', 'Syntax', 'Vocabulary', 'Phraseology', 'Grammar', 'Conventions']):
    class_preds = class_preds * 4 + 1
    class_labels = class_labels * 4 + 1

    class_preds = np.round(class_preds * 2) / 2
    class_labels = np.round(class_labels * 2) / 2

    class_preds = ((class_preds - 1) * 2).astype(int)
    class_labels = ((class_labels - 1) * 2).astype(int)

    qwk_scores = {}
    for i, aspect in enumerate(aspects):
        qwk_score = cohen_kappa_score(class_preds[:, i], class_labels[:, i], weights='quadratic')
        qwk_scores[aspect] = qwk_score
        print(f'QWK for {aspect}: {qwk_score:.4f}')

    return qwk_scores

In [14]:
for layer in model.deberta.encoder.layer[:9]:
    for param in layer.parameters():
        param.requires_grad = False
for param in model.deberta.embeddings.parameters():
    param.requires_grad = False

In [ ]:
train_losses = []
qwk_scores = []
best_qwk = -float('inf')

model.train()
for epoch in range(10):
    loss_per_epoch = 0
    loop = tqdm(train_loader, desc=f"Epoch {epoch+1}")

    all_preds = []
    all_targets = []

    for batch in loop:
        optimizer.zero_grad()
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        reg_target = batch["scores"].to(device)
        reg_preds = model(input_ids, attention_mask)
        loss = 0
        for i in range(7):
            loss += combined_loss(reg_preds[:, i], reg_target[:, i])
        loss /= 7
        loss_per_epoch += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=10)
        optimizer.step()
        scheduler.step()

    avg_loss = loss_per_epoch / len(train_loader)
    train_losses.append(avg_loss)
    print(f'Loss_per_epoch = {avg_loss}')

    model.eval()
    loop = tqdm(val_loader, desc=f"Epoch {epoch+1}")
    with torch.no_grad():
        for batch in loop:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            reg_target = batch["scores"].to(device)
            reg_preds = model(input_ids, attention_mask)
            all_preds.append(reg_preds)
            all_targets.append(reg_target)
    all_preds = torch.cat(all_preds, dim=0).detach().cpu().numpy()
    all_targets = torch.cat(all_targets, dim=0).detach().cpu().numpy()
    qwk = evaluate_qwk_for_aspects(all_preds, all_targets, aspects=['Overall', 'Cohesion', 'Syntax', 'Vocabulary', 'Phraseology', 'Grammar', 'Conventions'])
    qwk_scores.append(qwk)
    avg_qwk = sum(qwk.values()) / len(qwk)
    if avg_qwk - best_qwk > 1e-6:
        best_qwk = avg_qwk
        print(f"New best QWK: {best_qwk:.4f}")
        torch.save(model.state_dict(), "/kaggle/working/best_first_model.pth")
    model.train()

In [15]:
model = DebertaNHeads()
model.load_state_dict(torch.load('/kaggle/working/best_first_model.pth', weights_only=False))
model.to(device)

DebertaNHeads(
  (deberta): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(128002, 768, padding_idx=0)
      (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0-11): 12 x DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=768, out_features=768, bias=True)
              (key_proj): Linear(in_features=768, out_features=768, bias=True)
              (value_proj): Linear(in_features=768, out_features=768, bias=True)
              (pos_dropout): Dropout(p=0.1, inplace=False)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): DebertaV2SelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-0

In [ ]:
model.eval()
loop = tqdm(test_loader)

all_preds = []
all_targets = []

with torch.no_grad():
    for batch in loop:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        target = batch["scores"].to(device)
        preds = model(input_ids, attention_mask)
        all_preds.append(preds)
        all_targets.append(target)
all_preds = torch.cat(all_preds, dim=0).detach().cpu().numpy()
all_targets = torch.cat(all_targets, dim=0).detach().cpu().numpy()
qwk = evaluate_qwk_for_aspects(all_preds, all_targets, aspects=['Overall', 'Cohesion', 'Syntax', 'Vocabulary', 'Phraseology', 'Grammar', 'Conventions'])

In [16]:
df_add_cont = pd.read_csv('/kaggle/input/dress-d/DREsS_CASE_content.tsv',
                          sep='\t', header=0, index_col=0)
df_add_lang = pd.read_csv('/kaggle/input/dress-d/DREsS_CASE_language.tsv',
                          sep='\t', header=0, index_col=0)
df_add_org = pd.read_csv('/kaggle/input/dress-d/DREsS_CASE_organization.tsv',
                          sep='\t', header=0, index_col=0)
df_add_new = pd.read_csv('/kaggle/input/dress-d/DREsS_New.tsv',
                          sep='\t', header=0, index_col=0)
df_add_std = pd.read_csv('/kaggle/input/dress-d/DREsS_Std.tsv',
                          sep='\t', header=0, index_col=0)
df_add = pd.concat([df_add_cont,
                    df_add_lang,
                    df_add_org,
                    df_add_new,
                    df_add_std], ignore_index=True) 

In [17]:
df_add

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,prompt,essay,content,language,organization,total,source
0,Write about patience. Being patient means that...,@CAPS1 I'm here to prove you wrong and tell yo...,1.000000,NaN,NaN,NaN,NaN
1,Smoking should be completely banned at all the...,"So how can someone say 'No, you @MONTH1 not li...",1.000000,NaN,NaN,NaN,NaN
2,Write about patience. Being patient means that...,"In @CAPS1 middle school, in @PERSON2's dirty a...",1.000000,NaN,NaN,NaN,NaN
3,We all understand the benefits of laughter. Fo...,One of the most oddest things in life is how p...,1.000000,NaN,NaN,NaN,NaN
4,Write about patience. Being patient means that...,A time when I was patient was when I was helpi...,1.000000,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
48967,It is important for college students to have a...,I think that it is important for college stude...,4.166667,3.958333,4.583333,12.708333,ICNALE EE
48968,Smoking should be completely banned at all the...,I agree with the statement that smoking should...,3.750000,3.208333,3.750000,10.708333,ICNALE EE
48969,It is important for college students to have a...,"In my opinion, I am strongly agree with the id...",3.333333,3.333333,2.916667,9.583333,ICNALE EE
48970,It is important for college students to have a...,A part time job means to work in separate time...,1.666667,1.875000,1.666667,5.208333,ICNALE EE


In [ ]:
df_add = pd.read_csv('/kaggle/input/ellipse-combined/asap_1_2.csv', index_col=0)

In [ ]:
df_add = df_add[df_add['essay_set'].isin([1, 2])].reset_index()

In [ ]:
df_add = df_add['essay']

In [ ]:
df_add = df_add.rename('full_text')

In [22]:
texts = df_add['essay']
texts = texts.dropna()

In [23]:
pseudo = pd.concat([train_df['full_text'], texts], ignore_index=True)

In [24]:
class UnlabeledDataset(Dataset):
    def __init__(self, dataframe, tokenizer):
        self.df = dataframe
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        text = row.tolist()


        inputs = self.tokenizer(text,
                                padding="max_length",
                                truncation=True,
                                return_tensors="pt",
                                max_length=512)

        return {
            "input_ids": inputs["input_ids"],
            "attention_mask": inputs["attention_mask"]
        }

In [25]:
add_dataset = UnlabeledDataset(pseudo, tokenizer)
add_loader = DataLoader(add_dataset, batch_size=8)

In [26]:
model.eval()
loop = tqdm(add_loader)
all_preds = []
with torch.no_grad():
    for batch in loop:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        preds = model(input_ids, attention_mask)
        all_preds.append(preds)
all_preds = torch.cat(all_preds, dim=0).detach().cpu().numpy()

100%|██████████| 6585/6585 [21:57<00:00,  5.00it/s]


In [27]:
columns=["Overall",
         "Cohesion",
         "Syntax",
         "Vocabulary",
         "Phraseology",
         "Grammar",
         "Conventions"]

In [28]:
pseudo_df = pd.DataFrame(data=list(zip(*all_preds.T)), columns=columns)

In [29]:
pseudo_df['full_text'] = pseudo.values

In [30]:
for col in columns:
    for i in range(len(train_df)):
        pseudo_df.loc[i, col] = (pseudo_df.loc[i, col] + train_df.reset_index().loc[i, col]) / 2

/tmp/ipykernel_145/2701176831.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.6066778600215912' has dtype incompatible with float32, please explicitly cast to a compatible dtype first.
  pseudo_df.loc[i, col] = (pseudo_df.loc[i, col] + train_df.reset_index().loc[i, col]) / 2
/tmp/ipykernel_145/2701176831.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.6528847515583038' has dtype incompatible with float32, please explicitly cast to a compatible dtype first.
  pseudo_df.loc[i, col] = (pseudo_df.loc[i, col] + train_df.reset_index().loc[i, col]) / 2
/tmp/ipykernel_145/2701176831.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.5361486375331879' has dtype incompatible with float32, please explicitly cast to a compatible dtype fir

In [31]:
class PseudoDataset(Dataset):
    def __init__(self, dataframe, tokenizer):
        self.df = dataframe
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        text = row["full_text"].to_list()


        inputs = self.tokenizer(text,
                                padding="max_length",
                                truncation=True,
                                return_tensors="pt",
                                max_length=512)

        scores = torch.tensor(row[columns].values, dtype=torch.float32)

        return {
            "input_ids": inputs["input_ids"],
            "attention_mask": inputs["attention_mask"],
            "scores": scores
        }

In [32]:
combined_dataset = PseudoDataset(pseudo_df, tokenizer)
combined_loader = DataLoader(combined_dataset, batch_size=8, shuffle=True)

In [33]:
model_2 = DebertaNHeads()

In [34]:
for layer in model_2.deberta.encoder.layer[:9]:
    for param in layer.parameters():
        param.requires_grad = False
for param in model_2.deberta.embeddings.parameters():
    param.requires_grad = False

In [35]:
num_training_steps = len(combined_loader) * 10

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_2.to(device)
optimizer = optim.AdamW(model_2.parameters(), lr=4e-5)
scheduler = CosineAnnealingLR(optimizer, T_max=num_training_steps)

In [38]:
train_losses = []
qwk_scores = []
best_qwk = -float('inf')

model_2.train()
for epoch in range(10):
    loss_per_epoch = 0
    loop = tqdm(combined_loader, desc=f"Epoch {epoch+1}")

    all_preds = []
    all_targets = []

    for batch in loop:
        optimizer.zero_grad()
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        reg_target = batch["scores"].to(device)
        reg_preds = model_2(input_ids, attention_mask)
        loss = 0
        for i in range(7):
            loss += combined_loss(reg_preds[:, i], reg_target[:, i])
        loss /= 7
        loss_per_epoch += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model_2.parameters(), max_norm=10)
        optimizer.step()
        scheduler.step()

    avg_loss = loss_per_epoch / len(train_loader)
    train_losses.append(avg_loss)
    print(f'Loss_per_epoch = {avg_loss}')

    model_2.eval()
    loop = tqdm(val_loader, desc=f"Epoch {epoch+1}")
    with torch.no_grad():
        for batch in loop:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            reg_target = batch["scores"].to(device)
            reg_preds = model_2(input_ids, attention_mask)
            all_preds.append(reg_preds)
            all_targets.append(reg_target)
    all_preds = torch.cat(all_preds, dim=0).detach().cpu().numpy()
    all_targets = torch.cat(all_targets, dim=0).detach().cpu().numpy()
    qwk = evaluate_qwk_for_aspects(all_preds, all_targets, aspects=['Overall', 'Cohesion', 'Syntax', 'Vocabulary', 'Phraseology', 'Grammar', 'Conventions'])
    qwk_scores.append(qwk)
    avg_qwk = sum(qwk.values()) / len(qwk)
    if avg_qwk - best_qwk > 1e-6:
        best_qwk = avg_qwk
        print(f"New best QWK: {best_qwk:.4f}")
        torch.save(model_2.state_dict(), "/kaggle/working/best_second_model.pth")
    model_2.train()

Epoch 1:   0%|          | 2/6585 [00:01<1:10:31,  1.56it/s]


KeyboardInterrupt: 

In [37]:
model_2 = DebertaNHeads()
model_2.load_state_dict(torch.load('/kaggle/working/best_second_model.pth', weights_only=True))
model_2.to(device)

DebertaNHeads(
  (deberta): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(128002, 768, padding_idx=0)
      (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0-11): 12 x DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=768, out_features=768, bias=True)
              (key_proj): Linear(in_features=768, out_features=768, bias=True)
              (value_proj): Linear(in_features=768, out_features=768, bias=True)
              (pos_dropout): Dropout(p=0.1, inplace=False)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): DebertaV2SelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-0

In [39]:
model_2.eval()
loop = tqdm(test_loader)

all_preds = []
all_targets = []

with torch.no_grad():
    for batch in loop:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        target = batch["scores"].to(device)
        preds = model_2(input_ids, attention_mask)
        all_preds.append(preds)
        all_targets.append(target)
all_preds = torch.cat(all_preds, dim=0).detach().cpu().numpy()
all_targets = torch.cat(all_targets, dim=0).detach().cpu().numpy()
qwk = evaluate_qwk_for_aspects(all_preds, all_targets, aspects=columns)

100%|██████████| 163/163 [00:32<00:00,  5.02it/s]


QWK for Overall: 0.7446
QWK for Cohesion: 0.6384
QWK for Syntax: 0.6733
QWK for Vocabulary: 0.6582
QWK for Phraseology: 0.6680
QWK for Grammar: 0.7125
QWK for Conventions: 0.6926
